<a href="https://colab.research.google.com/github/aicuai/Fooocus/blob/main/Fooocus_onColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fooocus 日本語アニメ特化版 [j.aicu.ai/FoooC](https://j.aicu.ai/FoooC)

Version 20240527

このコードは midJourneyや nijiJourneyスタイルのカンタン高画質のAI画像生成が行える「Fooocus」をAICU社が原作そのままに Google Colab (無料も一部可) を使って、日本語版で動くようにしたものです。

配布等は原作のライセンスに従います。

https://github.com/lllyasviel/Fooocus/blob/main/fooocus_colab.ipynb

- [AICU社によるFooocus解説記事](https://corp.aicu.ai/hs-search-results?term=Fooocus)
- [このスクリプトを管理しているリポジトリ](https://github.com/aicuai/Artist-Guide-for-SDXL/)
- [日本語化プロジェクトリポジトリ](https://github.com/aicuai/Fooocus/blob/ja/language/ja.json) 貢献したいひといらっしゃいますか？

☆なお日本語版翻訳にあたっては「**小学生でもお年寄りでもわかる日本語**」というコンセプトで翻訳しています。なぜなら中途半端なカタカナ英語でもわかる方や、CGの知識がある方々なら英語原作版を使えばよいからです。より多くの人々に Fooocus を使っていただきたい気持ちで翻訳しています。

## 超初心者向け使い方

ランタイムを実行「▶」を押してしばらくお待ち下さい。 https://....gradio.com のURLが表示されたらそれをクリックして別タブで遊べます。最大72時間有効ですが、途中で切断されたり止まったり、エラーが出た場合は「■」（停止）をしてもう一度「▶」を押してください。

## 起動オプション

このスクリプトではGUIで操作できるスイッチにしてあります。

## Google Driveへの保存について

オリジナルの Fooocus には何故か指定ディレクトリに保存する機能がないのですが、このスクリプトでは Google Driveの マイドライブ直下「Fooocus-outputs」に保存されるようにしました。これでセッションが切れてもだいじょうぶ！

## 無料版 Colab での動作について
このColabはデフォルトでrefinerを無効にすることに注意してください。なぜならColab freeのリソースは比較的限られているからです(そして画像プロンプトのようないくつかの「大きな」機能はfree-tier Colabの切断を引き起こすかもしれません)。フリー版のColabでは、基本的なテキストから画像への変換が常に動作するようにしています。

→確認したところ Image Prompt 「画像で指示」は Google Colab Pro T4 GPU では動きませんでした。

In [ ]:
import shutil
from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/gdrive')

target_drive_dir = "/content/gdrive/MyDrive/Fooocus-outputs"  # @param {type:"string"}
target_local_dir = "/content/Fooocus/outputs"

# ローカル(GDrive)出力ディレクトリを作成
os.makedirs(target_local_dir, exist_ok=True)

# シンボリックリンクの作成
if os.path.exists(target_drive_dir):
    if os.path.islink(target_drive_dir):
        os.unlink(target_drive_dir)

os.symlink(target_local_dir, target_drive_dir)

# オプションの設定
use_japanese = True  # @param {type:"boolean"}
model_type = "anime"  # @param ["default", "anime"] {allow-input: true}
fixed_version = True  # @param {type:"boolean"}

# 必要なパッケージのインストール
!pip install pygit2==1.12.2

%cd /content

if fixed_version:
    if os.path.exists("Fooocus"):
        !rm -rf Fooocus
    !git clone https://github.com/lllyasviel/Fooocus.git
    %cd Fooocus
    !git checkout e2f9bcb
else:
    !git clone https://github.com/lllyasviel/Fooocus.git
    %cd Fooocus

# 日本語パッチを追加
if use_japanese:
    ja_json_url = "https://raw.githubusercontent.com/aicuai/Fooocus/ja/language/ja.json"
    ja_json_path = "/content/Fooocus/language/ja.json"
    os.makedirs(os.path.dirname(ja_json_path), exist_ok=True)
    !curl -L -o {ja_json_path} {ja_json_url}
    print(f"ja.json downloaded and saved to {ja_json_path}")

# アニメ版の設定ファイルのパッチ(最新版)を追加
if model_type == "anime":
    anime_json_url = "https://raw.githubusercontent.com/aicuai/Fooocus/main/presets/anime.json"
    anime_json_path = "/content/Fooocus/presets/anime.json"
    os.makedirs(os.path.dirname(anime_json_path), exist_ok=True)
    !curl -L -o {anime_json_path} {anime_json_url}
    print(f"anime.json downloaded and saved to {anime_json_path}")

%cd /content/Fooocus

# Fooocus JP Script Options
if model_type == "default" and not use_japanese:
    !python entry_with_update.py  --always-high-vram --share
elif model_type == "default" and use_japanese:
    !python entry_with_update.py  --always-high-vram --share --language ja
elif model_type == "anime" and not use_japanese:
    !python entry_with_update.py --preset anime --always-high-vram --share
elif model_type == "anime" and use_japanese:
    !python entry_with_update.py --preset anime --output-path {target_drive_dir} --always-high-vram --share --language ja


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @markdown # [使用後]生成した画像を圧縮して退避します
!rm /content/outputs.zip
%cd /content/Fooocus/
!zip -r  ../outputs.zip ./outputs
# @markdown ### Google Drive のマイドライブ直下に「Fooocus-outputs」フォルダに展開します

MyDrive_Path = "/content/drive/MyDrive/Fooocus-outputs" # @param {type:"string"}

# @markdown  **空白** にして実行すると直接ダウンロードします。
from google.colab import files

if MyDrive_Path!="" :
  !unzip -o /content/outputs.zip -d $MyDrive_Path
else:
  files.download('/content/outputs.zip')


